In [171]:
%run setup_path.py
%load_ext autoreload
%autoreload 2

from os import path

from gorbslam.orbslam_processor import ModelType, ORBSLAMProcessor

# TRAJECTORY_NAME = 'vabadussild'
# TRAJECTORY_NAME = 'vabadusepst'
# TRAJECTORY_NAME = 'oldtown'
TRAJECTORY_NAME = 'vabadussild_novatel'
# TRAJECTORY_NAME = 'vabadusepst_novatel'
# TRAJECTORY_NAME = 'oldtown_novatel'
# TRAJECTORY_NAME = 'kitti/09_30_drive_0018'

model_type = ModelType.NN
# model_type = ModelType.NN_CLR
# model_type = ModelType.GBR
# model_type = ModelType.SVR
# model_type = ModelType.RFR
# model_type = ModelType.UMEYAMA

processor = ORBSLAMProcessor(path.join('~/orbslam-out', TRAJECTORY_NAME), model_type)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# processor.initialize_model(overwrite=True)
processor.initialize_model()


In [ ]:
processor.fit_trajectories()


In [ ]:
processor.save_trajectories()


In [ ]:
processor.create_ape_plot_all().show()


In [ ]:
processor.create_ape_plot().show()


In [ ]:
processor.create_ape_plot(0).show()


In [ ]:
# processor.create_2d_plot_utm().show()


In [ ]:
# processor.create_2d_plot_slam().show()


In [ ]:
# processor.create_map_plot().show()


In [ ]:
# processor.create_3d_plot_slam().show()
